#Expenditure Type vs Access Type

##General Code

In [166]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pysentani
import scipy as sp

In [167]:
survey = pd.read_excel('../data-clean/sentani-merged-cleaned-2015-06-10.xlsx',na_values=[''])
print('number of entries =', len(survey))
print('number of columns =', len(survey.columns))

number of entries = 1184
number of columns = 274


In [203]:
survey['access_type'] = pysentani.access_type(survey)
pysentani.similar_columns(survey,'app_now')

app_now
app_now/lighting
app_now/TV
app_now/radio
app_now/fridge
app_now/fan
app_now/rice_cooker
app_now/other_cooking
app_now/welder
app_now/grinder
app_now/saw
app_now/other_tools
app_now/other


##Raw access and expenditure data

In [169]:
survey[['access_type','PLN_expenditure','community_microgrid_expenditure','genset_expenditure']]

,access_type,PLN_expenditure,community_microgrid_expenditure,genset_expenditure
0,no_access,NaN,NaN,50000
1,no_access,NaN,NaN,NaN
2,no_access,NaN,NaN,NaN
3,no_access,NaN,NaN,NaN
4,no_access,NaN,NaN,20000
5,no_access,NaN,NaN,30000
6,no_access,NaN,NaN,20000
7,no_access,NaN,NaN,NaN
8,no_access,NaN,NaN,NaN
9,no_access,NaN,NaN,20000


##No Access 

###Expected to correspond to genset_expenditure

In [170]:
no_access_mask = survey['access_type'] == 'no_access'
#access_survey = temp_access_survey[pd.notnull(temp_access_survey['community_microgrid_expenditure']) | (pd.notnull(temp_access_survey['PLN_expenditure']))
cp_exp_mask = (pd.notnull(survey['community_microgrid_expenditure']) |
            pd.notnull(survey['PLN_expenditure'])
            )

survey['sim_key'] = survey['simserial'].str[-5:]
survey[no_access_mask & cp_exp_mask][['village_name',
               'access_type',
               'PLN_expenditure',
               'community_microgrid_expenditure',
               'genset_expenditure',
               'demand_point',
               'sim_key']].head()

,village_name,access_type,PLN_expenditure,community_microgrid_expenditure,genset_expenditure,demand_point,sim_key
137,Yokiwa,no_access,NaN,240000,NaN,school,5028f
287,Puai,no_access,NaN,500000,NaN,school,5028f
288,Puai,no_access,NaN,100000,NaN,household,5028f
298,Puai,no_access,NaN,60000,NaN,household,5028f
306,Yokiwa,no_access,NaN,300000,NaN,household,5028f


##Community Microgrid Access

###Expected to correspond to community_microgrid_expenditure

In [171]:
community_access_mask = survey['access_type'] == 'community_microgrid'
#access_survey = temp_access_survey[pd.notnull(temp_access_survey['community_microgrid_expenditure']) | (pd.notnull(temp_access_survey['PLN_expenditure']))
gp_exp_mask = (pd.notnull(survey['genset_expenditure']) |
            pd.notnull(survey['PLN_expenditure'])
           )
survey[community_access_mask & gp_exp_mask][['village_name',
               'access_type',
               'PLN_expenditure',
               'community_microgrid_expenditure',
               'genset_expenditure',
               'demand_point',
               'sim_key']]

,village_name,access_type,PLN_expenditure,community_microgrid_expenditure,genset_expenditure,demand_point,sim_key
126,Kensio,community_microgrid,34000,NaN,NaN,household,5036f
152,Atamali,community_microgrid,NaN,NaN,150000,church,5028f


##PLN Grid Access

###Expected to correspond to PLN_expenditure and possibly genset_expenditure

(Thus this is the only category with no clearly anomalous data)

In [172]:
pln_access_mask = survey['access_type'] == 'PLN_grid'
#access_survey = temp_access_survey[pd.notnull(temp_access_survey['community_microgrid_expenditure']) | (pd.notnull(temp_access_survey['PLN_expenditure']))
cg_exp_mask = (pd.notnull(survey['community_microgrid_expenditure']) |
            pd.notnull(survey['genset_expenditure'])
           )
survey[pln_access_mask & cg_exp_mask][['village_name',
               'access_type',
               'PLN_expenditure',
               'community_microgrid_expenditure',
               'genset_expenditure',
               'demand_point',
               'sim_key']]

,village_name,access_type,PLN_expenditure,community_microgrid_expenditure,genset_expenditure,demand_point,sim_key
120,Hobong,PLN_grid,250000,NaN,500000,church,5036f
865,Khamayakha,PLN_grid,720000,NaN,50000,household,1217f
1120,Khamayakha,PLN_grid,4800000,NaN,45000,other,4027f
